# Random Forest Classifier implementation with pipelines and Hyperparameter tuning

In [2]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [5]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [6]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [8]:
df['time'].value_counts()

0    176
1     68
Name: time, dtype: int64

In [13]:
X=df.drop(labels=['time'],axis=1)
y=df['time']

In [15]:
X.head()


,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [16]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: time, dtype: int64

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [20]:
from sklearn.impute import SimpleImputer ## for missing value haldling
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [21]:
categorical_cols=['sex','smoker','day']
numerical_cols=['total_bill','tip','size']

In [23]:
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),#for missing value
    ('scaler',StandardScaler())   #feature scaling
    ]
)
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehotencoder',OneHotEncoder())
    ]
)

In [24]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [25]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [27]:
X_train

array([[-0.79306155, -0.2580329 , -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.46322744, -0.74211442, -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.80730659,  0.6399734 , -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.65383098, -1.46472887, -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.64749986,  0.32426806, -0.61214068, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.75289699, -0.41237773,  0.45363997, ...,  1.        ,
         0.        ,  0.        ]])

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [38]:
models={
    'Random Forest': RandomForestClassifier(),
    'Decision Tree' :DecisionTreeClassifier(),
    'SVC': SVC()
}

In [31]:
from sklearn.metrics import accuracy_score


In [39]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(X_train,y_train)
        y_test_pred=model.predict(X_test)
        test_model_score=accuracy_score(y_test,y_test_pred)
        report[list(models.keys())[i]] = test_model_score
    return report

In [40]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Decision Tree': 0.9387755102040817,
 'SVC': 0.9591836734693877}

In [47]:
classifier=RandomForestClassifier()
params={'max_depth':[3,4,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']
}

In [42]:
from sklearn.model_selection import RandomizedSearchCV

In [48]:
CV=RandomizedSearchCV(classifier,param_distributions=params,cv=5,scoring='accuracy',verbose=3)

In [49]:
CV.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.974 total time=   0.6s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.923 total time=   0.6s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=1.000 total time=   0.6s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.949 total time=   0.6s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.923 total time=   0.6s
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.974 total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.923 total time=   0.4s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=200;, score=1.000 total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.949 total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=200;, score

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 4, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [51]:
CV.best_params_

{'n_estimators': 200, 'max_depth': 10, 'criterion': 'entropy'}